In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.linear_model import LinearRegression
# Only use this if running the notebook on your local machine
#plt.style.use('notebook.mplstyle')

### Computational complexity
It is always useful to have at least a rough idea of the computational complexity of your data analysis. By this I mean the amount of computations that has to be performed and how this changes as a function of data collected. One common way to assess this is to consider how the amount of computations increase as your gather more data. However, one is rarely interested in the exact time taken (that depends on your hardware) but rather how the computations scales: does the computation time grow linearly ($\propto n^1$), quadratically ($\propto n^2$), cubically ($\propto n^3$), etc. For example, if the computation time grows cubicly, then it will take 1000 times longer to do your analysis if you collect ten times more data. Worst case scenario is then that you go through the trouble of collecting 10 times more data just to find out that you don't have the time and resources to analyze it. In this assignment you will get a little bit more familiar with computational complexity by analyzing how long it takes to compute the distance between all data points in your data set.

### Computational complexity for computing the distance between all pairs of data points
Assume you have a data matrix $\mathbf{X}$, where each row denotes one data point, for example row $i$ denotes the data point $\mathbf{x}_i$. The distance ($d_{ij}$) between data points $\mathbf{x}_i$ and $\mathbf{x}_j$ is found by generalizing Pythagorean theorem to higher dimensions as:
\begin{equation}
 d_{ij} = \sqrt{(x_{i1}-x_{j1})^2+x_{i2}-x_{j2})^2+...+(x_{im}-x_{jm})^2},
\end{equation}
where $m$ denotes the dimensionality of our data (the number of columns in $\mathbf{X}$).

To get started, let us define a function that computes a distance matrix ($D$) for a data matrix $\mathbf{X}$, where each elements $d_{ij}$ in $\mathbf{D}$ denotes the distance between data points $\mathbf{x}_i$ and $\mathbf{x}_j$ (that is row $i$ and $j$ in $\mathbf{X}$).

In [ ]:
def compute_distances(X):
    
    # Check how many rows (data points) X has
    n_data_points = X.shape[0]
    # Create a matric to store the distance values in
    D = np.zeros([n_data_points, n_data_points])
    
    # First loop over all data points
    for i in range(n_data_points):
        # Select the data point from corresponding to row i
        xi = X[i, :]
        
        # Second loop over all data points
        for j in range(n_data_points):
            # Select the data point from corresponding to row i
            xj = X[j, :]
            # Compute the difference vector between the data points
            x_diff = xi - xj
            # Compute the distance of the difference vector
            d_tmp = np.sqrt( np.sum( x_diff**2 ) )
            # Store the distance value into our distance matrix
            D[i, j] = d_tmp
            
    return D

Before doing anything else, it is a good idea to check that our function is doing what we want. So lets test it on a small matrix $\mathbf{X}$ for which we can visually inspect the result.

In [ ]:
# Randomly gerenate a few 2-dimensional data points
X = np.random.rand(5, 2)

# Compute the distance between each pair of data points
D = compute_distances(X)
# Print the matrix
print(D)

# Inspect the computed values visually
# Check that the distances values for data points far apart are 
# large, and wise-versa.
fig, ax = plt.subplots(1, 1)
ax.plot(X[:, 0], X[:, 1], 'o', alpha=0.5)
for i in range(5):
    ax.text(X[i, 0], X[i, 1], str(i+1), va='bottom', ha='left')
ax.set(xlabel='$x_1$', ylabel='$x_2$', xlim=[0, 1], ylim=[0, 1], 
       title='Randomly generated data points');

### Generating data
Now that our function works, we can start analyzing the time required to compute $\mathbf{D}$ as a function of the amount of data (number of rows in $\mathbf{X}$). To do so, we simply generate random data matrices with varying number of rows (denoted by $n$ below) and measure how long it takes to compute $\mathbf{D}$ for various $n$ values.

In [ ]:
# The dimensionality of the randomly generated data
m = 100
# Select the number of data points to measure the computation time for
# Modify if needed so that the time required for the larges n value is a few seconds.
n_values = 20*2**np.arange(6)

# Loop over all n values to try
comp_times_s = []
for n_tmp in n_values:
    
    # Genrerate a random data matric with n_tmp rows and m columns
    X_tmp = np.random.rand(n_tmp, m)
    # Measure the computation time, see 
    # https://ipython.readthedocs.io/en/stable/interactive/magics.html
    # for how to use the timeit magic command
    result = %timeit -n1 -r1 -o -q compute_distances(X_tmp)
    # Store the average time taken to evaluate the function
    comp_times_s.append(result.best)
    
    print('n = {:1.5g}: {:1.3f}'.format(n_tmp, result.best))

As always, once we have some data, we start off by looking at it.

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(n_values, comp_times_s, 'o:', alpha=0.5)
ax.set(xlabel='Number of data points', ylabel='Computation time (s)', 
       title='Distance computation between all data points');

The computation time clearly grows as $n$ increases, and we also note that it definitely grows faster than just linearly, the question is just how much faster? The expectation is that we can model the relationship using a model of the form:
\begin{equation}
    t = c n^k,
\end{equation}
where $t$ is the time required, $c$ the time required for one data point, and $k$ an exponent denoting how fast the growth is (1 for linear, 2 for quadratic, etc). This is not a linear model, but once we take the logarithm of both sides we get:
\begin{equation}
    \log{t} = \log c + k \log n,
\end{equation}
meaning that the logarithm of the time required is a linear function of the logarithm of $n$ (the number of data points), and that $k$ corresponds to the slope of this linear function.

### Your job is to:
1. Take the logarithm (np.log10) of the time required $t$ as well as $n$.
2. Plot the data again and verify that the relationship now appears to be linear.
3. Use linear regression to learn the value of $k$ from the data.
4. Discuss what the implications of the found $k$ value are in practice.
5. Try to explain (based on reasoning) what the $k$ value should be when computing the distances between each data point.